In [ ]:
import medmnist
from medmnist import INFO, Evaluator
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import sys

data_flag = 'chestmnist' 
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

img_size = 128

DataClass = getattr(medmnist, info['python_class'])
train_data = DataClass(split='train', download=True, size=img_size)
test_data = DataClass(split='test', download=True, size=img_size)
val_data = DataClass(split='val', download=True, size=img_size)

X_train, y_train = train_data.imgs, train_data.labels
X_val, y_val = val_data.imgs, val_data.labels
X_test, y_test = test_data.imgs, test_data.labels

X_train = X_train.astype('float16')
X_val = X_val.astype('float16')
X_test = X_test.astype('float16')


# Normalize images to [0, 1]
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

In [ ]:
# Convert one-hot encoded labels to class indices
y_train = np.argmax(y_train, axis=1)
y_val = np.argmax(y_val, axis=1)
y_test = np.argmax(y_test, axis=1)

In [ ]:
# Convert all classes to 1 or 0
def convert2Binary(npArr):
    for i in range(len(npArr)):
        if(npArr[i] != 0):
            npArr[i] = 1
    return npArr
y_train = convert2Binary(y_train)
y_val = convert2Binary(y_val)
y_test = convert2Binary(y_test)

In [ ]:
binary_classes = [0, 1]

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, n_channels)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = (
    train_dataset.shuffle(buffer_size=1000) 
    .batch(64)  
    .prefetch(buffer_size=tf.data.AUTOTUNE)  # Prefetch data to avoid bottlenecks
)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64)

'''
# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))
'''

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt

def build_model(hp):
    model = Sequential()

    model.add(Conv2D(
        filters=hp.Int('filters_1', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size_1', values=[3, 5]),
        activation='relu',
        input_shape=(img_size, img_size, 1)
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Conv2D(
        filters=hp.Int('filters_2', min_value=64, max_value=256, step=64),
        kernel_size=hp.Choice('kernel_size_2', values=[3, 5]),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Conv2D(
        filters=hp.Int('filters_3', min_value=128, max_value=512, step=128),
        kernel_size=hp.Choice('kernel_size_3', values=[3, 5]),
        activation='relu',
        padding='same'
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(hp.Float('dropout_3', min_value=0.3, max_value=0.6, step=0.1)))


    model.add(Flatten())
    model.add(Dense(
        units=hp.Int('dense_units', min_value=128, max_value=512, step=128),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout_dense', min_value=0.3, max_value=0.6, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
from keras_tuner import RandomSearch

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,  
    executions_per_trial=2, 
    directory='hyperparameter_search',
    project_name='cnn_tuning_optimized_highres'
)

In [ ]:
tuner.search(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    ]
)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Optimal Filters for Conv Layer 1: {best_hps.get('filters_1')}
Optimal Kernel Size for Conv Layer 1: {best_hps.get('kernel_size_1')}
Optimal Filters for Conv Layer 2: {best_hps.get('filters_2')}
Optimal Kernel Size for Conv Layer 2: {best_hps.get('kernel_size_2')}
Optimal Dense Units: {best_hps.get('dense_units')}
Optimal Learning Rate: {best_hps.get('learning_rate')}
""")

In [ ]:
best_model = tuner.hypermodel.build(best_hps)

history = best_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32
)

In [ ]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

y_pred = (best_model.predict(X_test) > 0.5).astype('int32')

from sklearn.metrics import classification_report, confusion_matrix
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()